In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [ ]:

configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "client_id",
"fs.azure.account.oauth2.client.secret": "secret_key",
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenant_id/oauth2/token"}


dbutils.fs.mount(
source = "abfss://tokyo-data@tokyoanalytics.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/tokyo-dat",
extra_configs = configs)

Out[19]: True

In [ ]:
%fs
ls "/mnt/tokyo-dat"

path,name,size,modificationTime
dbfs:/mnt/tokyo-dat/raw-data/,raw-data/,0,1692704135000
dbfs:/mnt/tokyo-dat/transform-data/,transform-data/,0,1692704148000


In [ ]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[*, 4] 
 AppName 
 Databricks Shell

In [ ]:
athletes = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyo-dat/raw-data/athletes.csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyo-dat/raw-data/medals.csv")
coaches = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyo-dat/raw-data/coaches.csv")
gender = spark.read.format("csv").option("header","true").load("/mnt/tokyo-dat/raw-data/genderentries.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyo-dat/raw-data/teams.csv")

In [ ]:
teams.show()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [ ]:
athletes.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [ ]:
gender = gender.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
        .withColumn("Total",col("Total").cast(IntegerType()))

In [ ]:
gender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [ ]:
medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Team_Country: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [ ]:
# Find the top countries with highest number of gold medals
# Specific processing
top_gold_medals_countries = medals.orderBy("Gold",ascending=False).select("Team_Country","Gold").show()

+--------------------+----+
|        Team_Country|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
|              Canada|   7|
|              Brazil|   7|
|         New Zealand|   7|
|                Cuba|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|              Poland|   4|
|      Czech Republic|   4|
|               Kenya|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows



In [ ]:
average_entries_by_gender = gender.withColumn(
    'Avg_Female' , gender['Female'] / gender['Total']
).withColumn(
    'Avg_Male' , gender['Male'] / gender['Total']
)
average_entries_by_gender.show()

+--------------------+------+----+-----+-------------------+-------------------+
|          Discipline|Female|Male|Total|         Avg_Female|           Avg_Male|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    98|  98|  196|                0.5|                0.5|
|   Artistic Swimming|   105|   0|  105|                1.0|                0.0|
|           Athletics|   969|1072| 2041| 0.4747672709456149| 0.5252327290543851|
|           Badminton|    86|  87|  173|0.49710982658959535| 0.5028901734104047|
|   Baseball/Softball|    90| 144|  234|0.38461538461538464| 0.6153846153846154|
|          Basketball|   144| 144|  288|                0.5|                0.5|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   102

In [ ]:
# athletes.write.option("header","true").csv("/mnt/tokyo-dat/transform-data/athletes")
# athletes.write.mode("overwrite").option("header","true").csv("/mnt/tokyo-dat/transform-data/athletes")
athletes.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyo-dat/transform-data/athletes") 
# Partition the csv file

In [ ]:
teams.printSchema()

root
 |-- <!DOCTYPE html>: string (nullable = true)



In [ ]:
coaches.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyo-dat/transform-data/coaches")
gender.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyo-dat/transform-data/gender")
medals.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyo-dat/transform-data/medals")
teams.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyo-dat/transform-data/teams")